# What is Decorator?

**Higher order** (**currying**) functions that are used as wrappers to other functions. 

There are decorator classes too.

Decorators may either:

- modify run-time behaviour of decorated function

- modify decorated function object

# What Can You Do With Decorator?

- Pre-process function arguments

- Post-process return value and/or intercept exception(s)

- Modify function object, e.g. add attributes

- conditionally replace function object with another callable

etc.

# Decorator does ....

## Classic example - 2 tiered

In [ ]:
def wrapper(func):
    """ Simple decorator """
    print(f'Decorating {func.__name__}')
    def runner(*args, **kwargs):
        """ Simple runner """
        print(f'Executing {func.__name__}')
        res = func(*args, **kwargs)
        print(f'Executed {func.__name__}')
        return res
    return runner 

**How does it work?**
```python
def wrapper(func):
    """ Simple decorator """
    print(f'Decorating {func.__name__}')
    def runner(*args, **kwargs):
        """ Simple runner """
        print(f'Executing {func.__name__}')
        res = func(*args, **kwargs)
        print(f'Executed {func.__name__}')
        return res
    return runner 
```

In [ ]:
@wrapper
def dummy_func():
    """ Simple function """
    print(f'I am just here for POC')

In [ ]:
dummy_func()

**What happpens in decoration?**
```python
def wrapper(func):
    """ Simple decorator """
    print(f'Decorating {func.__name__}')
    def runner(*args, **kwargs):
        """ Simple runner """
        print(f'Executing {func.__name__}')
        res = func(*args, **kwargs)
        print(f'Executed {func.__name__}')
        return res
    return runner 
```

In [ ]:
def dummy_func1():
    """ further proof function """
    print(f'I am just here for further proof')
    
dummy_func1 = wrapper(dummy_func1)

In [ ]:
dummy_func1()

** You can even do that ** (Why?)
```python
def wrapper(func):
    """ Simple decorator """
    print(f'Decorating {func.__name__}')
    def runner(*args, **kwargs):
        """ Simple runner """
        print(f'Executing {func.__name__}')
        res = func(*args, **kwargs)
        print(f'Executed {func.__name__}')
        return res
    return runner 
```

In [ ]:
def dummy_func2():
    """ Another function """
    print(f'I am just here for another proof')
wrapper(dummy_func2)()

## Anything I should be aware of?

In [ ]:
print(dummy_func.__name__, dummy_func1.__name__, dummy_func2.__name__, sep='\n')

### Why?

```python
def wrapper(func):
    """ Simple decorator """
    print(f'Decorating {func.__name__}')
    def runner(*args, **kwargs):
        """ Simple runner """
        print(f'Executing {func.__name__}')
        res = func(*args, **kwargs)
        print(f'Executed {func.__name__}')
        return res
    return runner 
```

```python
@wrapper
def some_func(...):
....
```
is equivalent to 

```python
some_func = wrapper(some_func):
....
```


Thanks to Python feature called **closure** internal function *runner* preserves content created when a function is decorated.

**Solution** - use *functools.wraps* decorator

In [ ]:
from functools import wraps
def proper_wrapper(func):
    """ Simple decorator """
    print(f'Decorating {func.__name__}')
    @wraps(func)
    def runner(*args, **kwargs):
        """ Simple runner """
        print(f'Executing {func.__name__}')
        res = func(*args, **kwargs)
        print(f'Executed {func.__name__}')
        return res
    return runner 

@proper_wrapper
def another_cute_function():
    print('I should be properly decorated')

In [ ]:
another_cute_function()

In [ ]:
another_cute_function.__name__

## Replacement decorator?!

In [ ]:
def substitute_in_debug(func):
    if DEBUG:
        print(f'Replacing "{func.__name__}" with stub')
        return lambda *_, **__: 'I am a substitute'
    else:
        print(f'Returning the original "{func.__name__}"')
        return func

In [ ]:
DEBUG = True
@substitute_in_debug
def real_mccoy():
    return 'I am real McCoy'

In [ ]:
real_mccoy()

**What if we change the flag?**
```python
def substitute_in_debug(func):
    if DEBUG:
        print(f'Replacing "{func.__name__}" with stub')
        return lambda *_, **__: 'I am a substitute'
    else:
        Print
        return func
```

In [ ]:
DEBUG = False
real_mccoy() 

**Decorator code executed at definition of decorated function!!!**

In [ ]:
DEBUG = False
@substitute_in_debug
def real_real_mccoy():
    return 'I am real real McCoy'
real_real_mccoy()

# Let us proceed to the fun part

## Easily log function calls

Decorator adds pre- and post-processing

In [ ]:
import logging
from functools import wraps
logging.basicConfig(level=logging.DEBUG)
 
def logging_wrapper(func):
    @wraps(func)
    def logging_inner(*args, **kwargs):
        kwargs_formatted = [f'{k}={repr(v)}' for k, v in kwargs.items()]
        arg_string = ', '.join([repr(v) for v in args] + kwargs_formatted)
        call_line = f'Func call: {func.__name__}({arg_string})'
        try: 
            res = func(*args, **kwargs)
            logging.debug(f'{call_line} - returns {repr(res)}')
            return res
        except Exception as exc:
            logging.exception(call_line + ' caused exception!')
    return logging_inner

#### Now, decorate function of your choice


In [ ]:
@logging_wrapper
def add(x, y):
    return x + y

#### Shall we test?  


In [ ]:
add(1, 4)

In [ ]:
add(1, '4')

#### But what about keyword arguments?


In [ ]:
@logging_wrapper
def add(first, second=None):
    return first + second

In [ ]:
add(1,4)

In [ ]:
add(1, second=4)


In [ ]:
add(1, second='4')

## Static variables

#### No such animal in Python?! 

#### Remember, you were told never to use default mutable values - but the language allows them?

In [ ]:
def count_calls(_counter=[0]):
    _counter[0] += 1
    print (f'I was called {_counter[0]} times')

In [ ]:
count_calls()

#### What about function attribute?

In [ ]:
def count_calls_take2():
    count_calls_take2.cnt += 1
    print (f'I was called {count_calls_take2.cnt} times')
setattr(count_calls_take2, 'cnt', 0)

In [ ]:
count_calls_take2()

It works, but it looks ugly. And if you need more than 1 "static" variable?

#### But what if you could do it in more elegant way?

Decorator modifies function object

In [ ]:
def static_variables(**static_kw):
    def static_setter(func):
        for static_name, init_value in static_kw.items():
            setattr(func, static_name, init_value)
        return func
    return static_setter

#### Now, just put it to work

In [ ]:
@static_variables(cnt=0)
def count_calls_take3():
    count_calls_take3.cnt +=1
    print(f'I was called {count_calls_take3.cnt} times')

In [ ]:
count_calls_take3()

## Plurals for grammar freaks

There's a great standard decorator _singledispatch_ that allows your to create dispatcher on function's first argument.

Sort of like function overloading.

Let us start by creating actual hard-working function that will try to define the proper plural form.

In [ ]:
import collections
from functools import singledispatch

def _make_plural(count, word):
    if count == 1:
        pass
    elif word[-1] in 'sxz' or word[-2:] in {'sh', 'ch'}:
        word += 'es'
    elif word[-1] == 'y' and word[-2] not in 'aoeui':
        word = word[:-1] + 'ies'
    elif word[-1] == 'f':
        word = word[:-1] + 'ves' 
    elif word[-2:] == 'fe':
        word = word[:-2] + 'ves'
    else:
        word += 's'
    return f'{count} {word}'

#### Next step - we want to be able to use it both on numbers and iterables - lists, e.g.

In [ ]:
@singledispatch
def plural(value, value_descr):
    return _make_plural(value, value_descr)

@plural.register(collections.Iterable)
def _(value, value_descr):
    return _make_plural(len(value), value_descr)

#### Will it work?

In [ ]:
print(plural(1, 'orange'),
      plural(2, 'apple'),
      plural(3, 'baby'),
      plural(4, 'match'),
      plural(5, 'leaf'),
      plural(6, 'knife'),
      plural(['apple', 'orange', 'grapefruit'], 'fruit'),
      sep='\n')

## His Royal Majesty - Singleton

Python is not big on **Design Patterns**. But.. you need it

There is a wealth of methods - few of them KISS-able.

There is an old Borg by [Alex Martinelli](http://code.activestate.com/recipes/66531-singleton-we-dont-need-no-stinkin-singleton-the-bo/)

```python
class Borg:
    __shared_state = {}
    def __init__(self):
        self.__dict__ = self.__shared_state
    # and whatever else you want in your class -- that's all!
```

Which essentially creates pseudo-singleton (sorry, not a fan)

But, with a simple 7-line decorator

In [ ]:
def singleton(cls):
    _seen = {}
    def instantiator(*args, **kwargs):
        if cls not in _seen:
            _seen[cls] = cls(*args, **kwargs)
        return _seen[cls]
    return instantiator

Let us create 2 different classes

In [ ]:
@singleton
class Example:
    pass

@singleton
class AnotherExample:
    pass

... and test, whether they are real singletons

In [ ]:
test_subject_1 = Example()
test_subject_2 = Example()
another_test_subject = AnotherExample()

print(test_subject_1 is test_subject_1, test_subject_1 is another_test_subject)

**It's alive!**

## Self organizing class

#### What kind of animal is that?

* You have an instance **self_organized** of a class **SelfOrganized** with methods **method_a**, **method_b**, etc.

* You need to call **a**,**b**, **c** and **d** in that order.

* You want to add new methods - **method_e** and **method_f** to the class - and add them to the execution list

* You want to make adding new methods to the class - and to the execution list - as hassle-free as possible

## The reason?

* From my career - adding methods displaying widgets on image

  * Widget list was growing

  * I had no desire to distribute widgets "manually" 

* Interview project with textual menu
    * I wanted to show off
    * ... and I did not get the job (for another reason)

## How?

 * Create a decorator for self-arranging methods ([SO credit](https://stackoverflow.com/a/12718272/1381627))

```python   
@staticmethod
def _make_menu_item(func, item_no=[0]):
    """ Method decorator
        Set func attribute `_item_idx` to mark decorated 
        function as menu item and use it to set order
    """
    item_no[0] += 1
    setattr(func, '_menu_idx', item_no[0])
    return func
_menu_enumerator = _make_menu_item.__func__
```

* Define methods to be part of self-arrangement

```python
@_menu_enumerator
def one(self, *args, **kwargs):
    print('You chose option 1. One is lonely.')

@_menu_enumerator
def two(self, *args, **kwargs):
    print('You chose option 2. Two\'s company.')
```

* Collect self-organizining methods
```python
def __init__(self):
    all_attribs = (getattr(self, attr) for attr in dir(self))
    menu_items = sorted((attr for attr in all_attribs 
                         if hasattr(attr, '_menu_idx')),
                        key=lambda a: a._menu_idx)
    self._menu = OrderedDict((str(m._menu_idx), m) 
                             for m in menu_items)
```

* Wrap function calls
```python
    def __call__(self):
        while True:
            print(self._menu_prompts)
            choice = input('Your choice? ')
            end_game = self._menu.get(choice, 
                                      self.bad_choice)()
            if end_game:
                break
```

### And now - the grand finale!

In [ ]:
from collections import OrderedDict

class AutoMenu:
    def __init__(self):
        """
        Menu is built by collecting functions decorated with "_menu_enumerator"
        """
        all_attribs = (getattr(self, attr) for attr in dir(self))
        menu_items = sorted((attr for attr in all_attribs if hasattr(attr, '_menu_idx')),
                             key=lambda a: a._menu_idx)
        self._menu_prompts = '\nMake your choice\n' + '\n'.join(
            '{:2d} - {}'.format(f._menu_idx, f.__name__.replace('_', ' ').title())
            for f in menu_items)
        self._menu = OrderedDict((str(m._menu_idx), m) for m in menu_items)

    @staticmethod
    def _make_menu_item(func, item_no=[0]):
        """ Method decorator
            Mark decorated function as menu item by setting attribute
            _item_idx and use it to set order
        """
        item_no[0] += 1
        setattr(func, '_menu_idx', item_no[0])
        return func

    _menu_enumerator = _make_menu_item.__func__

    @_menu_enumerator
    def one(self, *args, **kwargs):
        print('One is lonely.')

    @_menu_enumerator
    def two(self, *args, **kwargs):
        print('Two\'s company.')

    @_menu_enumerator
    def four(self, *args, **kwargs):
        print('Four is stuck in the middle!!!.')

    @_menu_enumerator
    def three(self, *args, **kwargs):
        print('Three\'s a crowd.')

    @_menu_enumerator
    def end_game(self, *args, **kwargs):
        print('You chose to exit. Farewell.')
        return True

    def bad_choice(self):
        print('Are you sure?')

    def __call__(self):
        while True:
            print(self._menu_prompts)
            choice = input('Your choice? ')
            end_game = self._menu.get(choice, self.bad_choice)()
            if end_game:
                break

In [ ]:
AutoMenu()()

For more fun stuff, see Facebook [Python Programming Language](https://www.facebook.com/groups/python.programmers/) group.